In [0]:
import sys
from pyspark.sql.functions import when
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
import pyspark.pandas as pp
import pgeocode
from scipy.optimize import linear_sum_assignment

In [0]:
args = getResolvedOptions(sys.argv,['s3_target_path_key','s3_target_path_bucket'])
bucket = args['s3_target_path_key']
fileName = args['s3_target_path_bucket']

print(bucket, fileName)

In [0]:
spark = SparkSession.builder.appName("CDC3").getOrCreate()

inputFilePath1 = f"s3a://cdc-output-pyspark/output/{fileName}"
inputFilePath2 = f"s3a://cdc-output-pyspark/output/{fileName}"

finalFilePath = f"s3a://cdc-output-pyspark/output"

In [0]:
# Changing to Pandas Dataframe
if "job" in fileName:
    # Header check!
    df_job0 = spark.read.option("header",True).csv(inputFilePath1)
    df_job = pp.DataFrame(df_job0)
if "worker" in fileName:
    # Header check!
    df_worker0 = spark.read.option("header",True).csv(inputFilePath2)
    df_worker = pp.DataFrame(df_worker0)

In [0]:
# df_worker: id, name, location
# df_job: id, name, location

zip_j = df_job["Location"]
zip_w = df_worker["Location"]

nzip = len(zip_j)
ntech = len(zip_w)


In [0]:
# UDF (User Defined Function)

In [0]:
dist = pgeocode.GeoDistance('US')

In [0]:
def costfn(zip_j,zip_w):
    
    #pd.dataframe
    zip_j_pd = pp.DataFrame(zip_j)
    zip_w_pd = pp.DataFrame(zip_w)
    
    nzip = len(zip_j_pd)
    ntech = len(zip_w_pd)
    
    dist = pgeocode.GeoDistance('US')

    costmat0 = np.empty((nzip,ntech))

    for x in range(ntech):
        wzip = np.array([zip_w_pd[x]]*nzip)
        costmat0[:,x] = dist.query_postal_code(wzip,zip_j)
    
    # two cases: nzip>ntech vs. nzip < ntech
    # first case: nzip> ntech

    if nzip > ntech:
        nrep = nzip//ntech
        nmodulo = nzip % ntech
        costmat = np.empty((nzip,1))
        for i in range(nrep):
            costmat = np.concatenate((costmat0,costmat),axis=1)

        costmat[:,nrep:] = costmat0[:nmodulo,:]
    
        row_idx, col_idx = linear_sum_assignment(costmat)
    
        res = row_idx

    if nzip < ntech:
        nrdd = ntech-nzip
        costadd = np.zeros((nrdd,ntech))
        costmat = np.concatenate((costmat0,costadd),axis=2)
    
        row_idx, col_idx = linear_sum_assignment(costmat)
    
        res = row_idx[0:nzip]
    
    return res